In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from __future__ import print_function
import imageio
from PIL import Image
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import zipfile

In [ ]:
  resnet = tf.keras.applications.ResNet50(
      include_top=True,
      weights="imagenet",
      classes=1000
  )
  resnet.trainable = False

In [ ]:
def model(input_shape = (224,224,3), classes = 2):
  X_input = tf.keras.Input((224,224,3))
  X = resnet(X_input,training = False)
  X = tf.keras.layers.Flatten()(X)
  X = tf.keras.layers.Dense(2,activation = tf.nn.softmax,name = 'end')(X)
  model = tf.keras.models.Model(inputs = X_input, outputs = X ,name = 'model_full')
  return model

In [ ]:
model = model(input_shape=(224,224,3),classes = 2)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.01 , epsilon = 0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")

In [ ]:
IMAGE_DIR = "../working/train"
filenames = os.listdir(IMAGE_DIR)
labels = [x.split(".")[0] for x in filenames]
df = pd.DataFrame({"filename": filenames, "label": labels})
df.head()

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function = tf.keras.applications.resnet50.preprocess_input)
train_generator = train_datagen.flow_from_dataframe(df, IMAGE_DIR,x_col="filename", y_col="label",
                                                    target_size=(224,224), class_mode="categorical",
                                                   batch_size=100, shuffle=True, seed=42)

In [ ]:
model.summary()

In [ ]:
hist = model.fit_generator(train_generator,epochs = 1)

In [ ]:
hist = model.fit_generator(train_generator,epochs = 1)

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

In [ ]:
IMAGE_DIR = "../working/test1"
filenames = os.listdir(IMAGE_DIR)
labels = [x.split(".")[0] for x in filenames]
df = pd.DataFrame({"filename": filenames, "label": labels})
df.head()